In [1]:
import boto3
import json
import time
from dotenv import load_dotenv
from datetime import datetime
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [5]:

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='TRIM_HORIZON'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)


In [6]:
# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details

shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start reading records
read_kinesis_records(stream_name, shard_iterator)

Received order: {'client_id': 330080946074, 'location': {'address': 'Pasaje Vicente Cortina, 73, Madrid, Spain', 'lat': 40.33355092987921, 'lon': -3.7755412253335807}, 'order_id': '12cb38cb-3540-4f71-99d5-c51959be7950', 'order': {'n_objects': 3, 'volume': 95.46738366727311, 'weight': 13.42331305572242}, 'status': 'in_progress', 'timestamp': '2024-09-29 18:52:47'}
Received order: {'client_id': 541110197413, 'location': {'address': 'Ronda de Ariel Pedro, 569, Madrid, Spain', 'lat': 40.343133212170876, 'lon': -3.7924130442969077}, 'order_id': 'f1616ac9-0acb-41e5-89a4-b6c38b3d09c1', 'order': {'n_objects': 3, 'volume': 6.142193665453858, 'weight': 47.271276129220766}, 'status': 'in_progress', 'timestamp': '2024-09-29 18:53:04'}
Received order: {'client_id': 502037643876, 'location': {'address': 'Acceso de Viviana Saldaña, 481, Madrid, Spain', 'lat': 40.53111085262664, 'lon': -3.6637699351963797}, 'order_id': 'a5795693-8910-42da-923b-ee3d09b251fb', 'order': {'n_objects': 7, 'volume': 16.0066

AccessDeniedException: An error occurred (AccessDeniedException) when calling the GetRecords operation: User: arn:aws:sts::277709186030:assumed-role/voclabs/user3002246=borja.gonzalez@alu.mbitschool.com is not authorized to perform: kinesis:GetRecords on resource: arn:aws:kinesis:us-east-1:277709186030:stream/OrderStreamForDispatching with an explicit deny in an identity-based policy